<a href="https://colab.research.google.com/github/astrovishalthakur/MachineLearning/blob/main/FeatureEngineering/Handling_Categorical_Data/NominalEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nominal Data

In statistics, nominal data (also known as nominal scale) is a type of data that is used to label variables without providing any quantitative value. ... Unlike ordinal data. One of the most notable features of ordinal data is that, nominal data cannot be ordered and cannot be measured.

<img src="https://cdn.corporatefinanceinstitute.com/assets/nominal-data.png" width="600">

In [1]:
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv("cars.csv")

In [9]:
df.head()

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


### We will apply onehotencoding only on fuel and owners today. Although brand is also an categorical column, we will not transform it for the sake of simplicity.

In [11]:
df["brand"].nunique()

32

In [12]:
df["fuel"].value_counts()

Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: fuel, dtype: int64

In [13]:
df.owner.value_counts()

First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: owner, dtype: int64

# One Hot Encoding using Pandas

In [14]:
pd.get_dummies(df, columns=["fuel", "owner"])

,brand,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,0,1,0,0,1,0,0,0,0
1,Skoda,120000,370000,0,1,0,0,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,0,0,1
3,Hyundai,127000,225000,0,1,0,0,1,0,0,0,0
4,Maruti,120000,130000,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,110000,320000,0,0,0,1,1,0,0,0,0
8124,Hyundai,119000,135000,0,1,0,0,0,1,0,0,0
8125,Maruti,120000,382000,0,1,0,0,1,0,0,0,0
8126,Tata,25000,290000,0,1,0,0,1,0,0,0,0


## To avoid problem of multicollinearity. We use

# K - 1 OneHotEncoding

In [15]:
pd.get_dummies(df, columns=["fuel", "owner"], drop_first=True)

,brand,km_driven,selling_price,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,1,0,0,0,0,0,0
1,Skoda,120000,370000,1,0,0,0,1,0,0
2,Honda,140000,158000,0,0,1,0,0,0,1
3,Hyundai,127000,225000,1,0,0,0,0,0,0
4,Maruti,120000,130000,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,110000,320000,0,0,1,0,0,0,0
8124,Hyundai,119000,135000,1,0,0,1,0,0,0
8125,Maruti,120000,382000,1,0,0,0,0,0,0
8126,Tata,25000,290000,1,0,0,0,0,0,0


For machine learning, you almost definitely want to use sklearn.OneHotEncoder. For other tasks like simple analyses, you might be able to use pd.get_dummies, which is a bit more convenient.

Note that sklearn.OneHotEncoder has been updated in the latest version so that it does accept strings for categorical variables, as well as integers.

The crux of it is that the sklearn encoder creates a function which persists and can then be applied to new data sets which use the same categorical variables, with consistent results.



```
from sklearn.preprocessing import OneHotEncoder

# Create the encoder.
encoder = OneHotEncoder(handle_unknown="ignore")
encoder.fit(X_train)    # Assume for simplicity all features are categorical.

# Apply the encoder.
X_train = encoder.transform(X_train)
X_test = encoder.transform(X_test)
```



Note how we apply the same encoder we created via X_train to the new data set X_test.

Consider what happens if X_test contains different levels than X_train for one of its variables. For example, let's say X_train["color"] contains only "red" and "green", but in addition to those, X_test["color"] sometimes contains "blue".

If we use pd.get_dummies, X_test will end up with an additional "color_blue" column which X_train doesn't have, and the inconsistency will probably break our code later on, especially if we are feeding X_test to an sklearn model which we trained on X_train.

And if we want to process the data like this in production, where we're receiving a single example at a time, pd.get_dummies won't be of use.

With sklearn.OneHotEncoder on the other hand, once we've created the encoder, we can reuse it to produce the same output every time, with columns only for "red" and "green". And we can explicitly control what happens when it encounters the new level "blue": if we think that's impossible, then we can tell it to throw an error with handle_unknown="error"; otherwise we can tell it to continue and simply set the red and green columns to 0, with handle_unknown="ignore".

# OneHotEncoding using Sklearn


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:4], df.iloc[:,-1], test_size=0.2, random_state=2)

In [23]:
y_test.head()

606      200000
7575     750000
7705    2000000
4305      52000
2685     509999
Name: selling_price, dtype: int64

In [21]:
X_test.head()

,brand,km_driven,fuel,owner
606,Hyundai,80000,Petrol,First Owner
7575,Mahindra,70000,Diesel,Second Owner
7705,Toyota,68089,Petrol,First Owner
4305,Hyundai,70000,Petrol,Second Owner
2685,Mahindra,97000,Diesel,Second Owner


In [24]:
from sklearn.preprocessing import OneHotEncoder

In [50]:
ohe = OneHotEncoder(drop="first", dtype=np.int32) # if we don't specify dtype, then categories will be set as float. e.g. 0.0

We want to apply one hot encoding on fuel and owner only. We need to apply onehotencoder seperately on both columns, then join them with dataset.

This can be solved by ColumnTransformer but for now let's go the hard way for the sake of learning OneHotEncoder.

In [51]:
X_train_new = ohe.fit_transform(X_train[["fuel", "owner"]]).toarray()

In [52]:
X_test_new = ohe.transform(X_test[["fuel", "owner"]]).toarray()

In [53]:
ohe.transform(X_train[["fuel", "owner"]])

<6502x7 sparse matrix of type '<class 'numpy.int32'>'
	with 8746 stored elements in Compressed Sparse Row format>

### If we create ohe object as 
# ohe = OneHotEncoder(drop="first", sparse=False)

### Then a numpy array is returned and we don't need to explicitly convert it to numpy array.

In [54]:
X_test_new

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [55]:
X_test_new.shape

(1626, 7)

In [56]:
X_train[["brand", "km_driven"]].values #numpy array

array([['Hyundai', 35000],
       ['Jeep', 60000],
       ['Hyundai', 25000],
       ...,
       ['Tata', 15000],
       ['Maruti', 32500],
       ['Isuzu', 121000]], dtype=object)

In [57]:
np.hstack((X_train[["brand", "km_driven"]].values, X_train_new))

array([['Hyundai', 35000, 1, ..., 0, 0, 0],
       ['Jeep', 60000, 1, ..., 0, 0, 0],
       ['Hyundai', 25000, 0, ..., 0, 0, 0],
       ...,
       ['Tata', 15000, 0, ..., 0, 0, 0],
       ['Maruti', 32500, 1, ..., 1, 0, 0],
       ['Isuzu', 121000, 1, ..., 0, 0, 0]], dtype=object)

In [58]:
np.hstack((X_train[["brand", "km_driven"]].values, X_train_new)).shape

(6502, 9)

# OneHotEncoding with Top Categories

In [60]:
counts = df["brand"].value_counts()

In [61]:
counts

Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Force               6
Land                6
Isuzu               5
Kia                 4
Ambassador          4
Daewoo              3
MG                  3
Opel                1
Peugeot             1
Ashok               1
Name: brand, dtype: int64

### We will create a category "uncommon" for brands having less than 100 cars.

In [62]:
threshold = 100
replace = counts[counts <= threshold].index

In [67]:
pd.get_dummies(df["brand"].replace(replace, "uncommon"), drop_first=True)

,Chevrolet,Ford,Honda,Hyundai,Mahindra,Maruti,Renault,Skoda,Tata,Toyota,Volkswagen,uncommon
0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,0,0,0,1,0,0,0,0,0,0,0,0
8124,0,0,0,1,0,0,0,0,0,0,0,0
8125,0,0,0,0,0,1,0,0,0,0,0,0
8126,0,0,0,0,0,0,0,0,1,0,0,0
